In [53]:
import os
import re
import json
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
from fake_useragent import UserAgent
import time
import random

# Configuration
BASE_DIR = "Files"
LISTINGS_CSV = os.path.join(BASE_DIR, "apartmentsPortugal.csv")
CONSOLIDATED_CSV = os.path.join(BASE_DIR, "Consolidated.csv")
BASE_URL = "https://www.imovirtual.com/comprar/apartamento/?page="
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Ensure directory exists
os.makedirs(BASE_DIR, exist_ok=True)

In [54]:
def scrape_page(page_num):
    """Scrape data from a single listing page with columns matching consolidated.csv"""
    url = f"{BASE_URL}{page_num}"
    response = requests.get(url, headers=HEADERS)
    
    if response.status_code != 200:
        print(f"Failed to retrieve page {page_num}. Status code: {response.status_code}")
        return None
    
    soup = BeautifulSoup(response.content, 'html.parser')
    articles = soup.find_all('article', {'data-cy': 'listing-item'})
    
    if not articles:
        return None
    
    # Initialize all columns present in consolidated.csv
    data = {
        'URL': [],
        'Titles': [],
        'Price': [],
        'Location': [],
        'Rooms': [],
        'Areas': [],
        'Bathrooms': [],
        'Listing Type': [],
        'Useful area': [],
        'Gross area': [],
        'Construction year': [],
        'Energetic certificate': [],
        'Enterprise': [],
        'DateScraped': [],
        'Page': [],
        'Description': []
    }
    
    scrape_date = datetime.now().strftime('%Y-%m-%d')
    
    for article in articles:
        # Existing fields
        data['URL'].append(
            f"https://www.imovirtual.com{article.find('a')['href']}" 
            if article.find('a') else np.nan
        )
        data['Titles'].append(
            article.find('p', class_='css-u3orbr').text.strip() 
            if article.find('p', class_='css-u3orbr') else np.nan
        )
        data['Price'].append(
            article.find('span', class_='css-2bt9f1').text.strip() 
            if article.find('span', class_='css-2bt9f1') else np.nan
        )
        data['Location'].append(
            article.find('p', class_='css-42r2ms').text.strip() 
            if article.find('p', class_='css-42r2ms') else np.nan
        )
        
        # Rooms and Areas
        dl = article.find('dl', class_='css-12dsp7a')
        if dl:
            dt_dd = {dt.text.strip(): dd.text.strip() 
                    for dt, dd in zip(dl.find_all('dt'), dl.find_all('dd'))}
            data['Rooms'].append(dt_dd.get('Tipologia', np.nan))
            area = dt_dd.get('Zona', np.nan)
            data['Areas'].append(area.split()[0] if area else np.nan)
        else:
            data['Rooms'].append(np.nan)
            data['Areas'].append(np.nan)
        
        # Empty columns to be filled in Part 2
        data['Bathrooms'].append(np.nan)
        data['Listing Type'].append(np.nan)
        data['Useful area'].append(np.nan)
        data['Gross area'].append(np.nan)
        data['Construction year'].append(np.nan)
        data['Energetic certificate'].append(np.nan)
        data['Enterprise'].append(np.nan)
        data['Description'].append(np.nan)
        data['DateScraped'].append(scrape_date)
        data['Page'].append(page_num)
    
    # Validate lengths
    lengths = [len(v) for v in data.values()]
    if len(set(lengths)) != 1:
        print(f"Data length mismatch in page {page_num}: {lengths}")
        return None
    
    return pd.DataFrame(data)

def scrape_listings(num_pages):
    """Scrape multiple listing pages with resume support."""
    if os.path.exists(LISTINGS_CSV):
        df_existing = pd.read_csv(LISTINGS_CSV)
        start_page = df_existing['Page'].max() + 1 if 'Page' in df_existing.columns else 1
    else:
        start_page = 1
    
    end_page = start_page + num_pages - 1
    session = requests.Session()
    
    for page in range(start_page, end_page + 1):
        print(f"Scraping page {page}...")
        df_page = scrape_page(page)
        if df_page is None:
            print(f"No data on page {page}. Stopping.")
            break
        
        # Save incrementally
        df_page.to_csv(LISTINGS_CSV, mode='a', header=not os.path.exists(LISTINGS_CSV), index=False)
        print(f"Page {page} saved.")


In [55]:
def scrape_details():
    """Scrape detailed information with corrected column names"""
    df = pd.read_csv(LISTINGS_CSV)
    
    # Identify URLs needing processing
    mask = df['Construction year'].isna() | df['Bathrooms'].isna()
    indices = df[mask].index.tolist()
    
    ua = UserAgent()
    session = requests.Session()
    
    for i, idx in enumerate(indices):
        try:
            url = df.loc[idx, 'URL']
            response = session.get(url, headers={'User-Agent': ua.random}, timeout=10)

            if response.status_code == 200:
                soup = BeautifulSoup(response.content, 'html.parser')

                # Get useful area (default to "N/A" if not available)
                df.at[idx, 'Useful area'] = "N/A"

                # Get gross area, rooms, and bathrooms
                try:
                    buttons = soup.find_all('button', class_='eezlw8k1 css-ds0a69')
                    if len(buttons) >= 3:
                        area_div = buttons[0].find('div', class_='css-1ftqasz')
                        df.at[idx, 'Gross area'] = area_div.get_text(strip=True) if area_div else "N/A"

                        rooms_div = buttons[1].find('div', class_='css-1ftqasz')
                        df.at[idx, 'Rooms'] = rooms_div.get_text(strip=True) if rooms_div else "N/A"

                        bathrooms_div = buttons[2].find('div', class_='css-1ftqasz')
                        df.at[idx, 'Bathroom'] = bathrooms_div.get_text(strip=True) if bathrooms_div else "N/A"
                    else:
                        df.at[idx, 'Gross area'] = "N/A"
                        df.at[idx, 'Rooms'] = "N/A"
                        df.at[idx, 'Bathroom'] = "N/A"
                except AttributeError:
                    df.at[idx, 'Gross area'] = "N/A"
                    df.at[idx, 'Rooms'] = "N/A"
                    df.at[idx, 'Bathroom'] = "N/A"

                # Get construction year
                try:
                    divs = soup.find_all('div', class_='css-t7cajz e15n0fyo1')
                    construction_year = "N/A"
                    for div in divs:
                        p_elements = div.find_all('p', class_='e15n0fyo2 css-nlohq6')
                        for i, p in enumerate(p_elements):
                            if "Ano de construção" in p.get_text(strip=True):
                                if i + 1 < len(p_elements):
                                    construction_year = p_elements[i + 1].get_text(strip=True)
                                break
                        if construction_year != "N/A":
                            break
                    df.at[idx, 'Construction year'] = construction_year
                except (AttributeError, IndexError):
                    df.at[idx, 'Construction year'] = "N/A"

                # Get energetic certificate
                try:
                    energetic_certificate = "N/A"
                    for div in divs:
                        p_elements = div.find_all('p', class_='e15n0fyo2 css-nlohq6')
                        for i, p in enumerate(p_elements):
                            if "Certificado energético" in p.get_text(strip=True):
                                if i + 1 < len(p_elements):
                                    energetic_certificate = p_elements[i + 1].get_text(strip=True)
                                break
                        if energetic_certificate != "N/A":
                            break
                    df.at[idx, 'Energetic certificate'] = energetic_certificate
                except (AttributeError, IndexError):
                    df.at[idx, 'Energetic certificate'] = "N/A"

                # Save progress every 50 URLs
                if (i + 1) % 50 == 0:
                    df.to_csv(LISTINGS_CSV, index=False)

                time.sleep(random.uniform(0.5, 1.5))

            else:
                print(f"Skipping {url} due to status code {response.status_code}")

        except Exception as e:
            print(f"Error processing {url}: {str(e)}")

    
    df.to_csv(LISTINGS_CSV, index=False)

In [56]:
def merge_data():
    """Merge data while preserving existing structure"""
    if os.path.exists(CONSOLIDATED_CSV):
        consolidated = pd.read_csv(CONSOLIDATED_CSV)
    else:
        consolidated = pd.DataFrame(columns=[
            'URL', 'Titles', 'Price', 'Location', 'Rooms', 'Areas',
            'Bathrooms', 'Listing Type', 'Useful area', 'Gross area',
            'Construction year', 'Energetic certificate', 'Enterprise',
            'DateScraped', 'Page', 'Description'
        ])
    
    new_data = pd.read_csv(LISTINGS_CSV)
    
    # Convert numeric columns
    numeric_cols = ['Construction year', 'Page']
    for col in numeric_cols:
        new_data[col] = pd.to_numeric(new_data[col], errors='coerce')
    
    # Merge and deduplicate
    combined = pd.concat([consolidated, new_data], ignore_index=True)
    combined = combined.drop_duplicates('URL', keep='last')
    
    # Remove temporary columns
    combined = combined.loc[:, ~combined.columns.str.contains('Unnamed')]
    
    combined.to_csv(CONSOLIDATED_CSV, index=False)
    print(f"Merged data saved. Total records: {len(combined)}")

In [57]:
scrape_listings(1)
scrape_details()
merge_data()

Scraping page 2...
Page 2 saved.


C:\Users\willi\AppData\Local\Temp\ipykernel_67424\2295770096.py:4: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  consolidated = pd.read_csv(CONSOLIDATED_CSV)


Merged data saved. Total records: 90076


In [58]:
combined = pd.read_csv("Files/Consolidated.csv", index_col=False)

C:\Users\willi\AppData\Local\Temp\ipykernel_67424\872192564.py:1: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  combined = pd.read_csv("Files/Consolidated.csv", index_col=False)


In [59]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90076 entries, 0 to 90075
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   URL                    90076 non-null  object 
 1   Titles                 90076 non-null  object 
 2   Price                  90076 non-null  object 
 3   Location               90076 non-null  object 
 4   Rooms                  90060 non-null  object 
 5   Areas                  90076 non-null  object 
 6   Bathrooms              81941 non-null  object 
 7   Listing Type           62115 non-null  object 
 8   Useful area            89747 non-null  object 
 9   Gross area             71747 non-null  object 
 10  Construction year      49756 non-null  float64
 11  Energetic certificate  89641 non-null  object 
 12  Enterprise             80494 non-null  object 
 13  DateScraped            76 non-null     object 
 14  Page                   76 non-null     float64
 15  De

In [60]:
combined.tail(5)

,URL,Titles,Price,Location,Rooms,Areas,Bathrooms,Listing Type,Useful area,Gross area,Construction year,Energetic certificate,Enterprise,DateScraped,Page,Description,Bathroom
90071,https://www.imovirtual.com/pt/anuncio/apartame...,Apartamento T2 no centro de Oliveira do Bairro,192 000 €,"Oliveira do Bairro, Oliveira do Bairro, Aveiro",NaN,84.95,NaN,NaN,NaN,NaN,2026.0,A,NaN,2025-01-28,2.0,NaN,NaN
90072,https://www.imovirtual.com/pt/anuncio/t3-a-5-m...,T3 a 5 Minutos da estaçao dos Barcos,274 000 €,"Barreiro e Lavradio, Barreiro, Setúbal",T3,100.0,NaN,NaN,NaN,100m²,NaN,D,NaN,2025-01-28,2.0,NaN,2
90073,https://www.imovirtual.com/pt/anuncio/andar-mo...,Andar Moradia T3 | Maia | Terraço | Box | Metr...,439 000 €,"Rua das Caleiras, Centro, Cidade da Maia, Maia...",T3,144.0,NaN,NaN,NaN,144m²,2005.0,B-,NaN,2025-01-28,2.0,NaN,2
90074,https://www.imovirtual.com/pt/anuncio/apartame...,Apartamento T1 | Em construção | Terraço | Var...,295 000 €,"Arca d'Água - Campo Lindo - Vale Formoso, Para...",NaN,55.0,NaN,NaN,NaN,NaN,2024.0,B,NaN,2025-01-28,2.0,NaN,NaN
90075,https://www.imovirtual.com/pt/anuncio/apartame...,Apartamento T3 para venda,325 000 €,Rua Doutor Alfredo Pires Miranda - Bairro do F...,T3,137.0,NaN,NaN,NaN,137m²,2025.0,A,NaN,2025-01-28,2.0,NaN,3
